In [7]:
#-*- coding: utf-8 -*-
# 라이브러리 import
try:
    from selenium import webdriver
except:
    !pip install selenium
    from selenium import webdriver

try:
    from webdriver_manager.chrome import ChromeDriverManager
except:
    !pip install webdriver_manager
    from webdriver_manager.chrome import ChromeDriverManager

try:
    from urllib.parse import urlencode
except:
    !pip install urllib3
    from urllib.parse import urlencode
import datetime
import time

try:
    import csv
except:
    !pip install csv
    import csv

In [8]:
# 브라우저 옵션
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('start-maximized')
options.add_argument(
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        )
options.add_argument("lang=ko_KR")

In [9]:
# 루프 제한 값
sleepMaxCount = 5
# select query
item_query = "ytd-video-renderer.style-scope.ytd-item-section-renderer"
title_query = "div#title-wrapper yt-formatted-string.style-scope.ytd-video-renderer"
like_query = "ytd-video-meta-block.style-scope.ytd-video-renderer.byline-separated > div#metadata > div#metadata-line > span"
time_query = "ytd-video-meta-block.style-scope.ytd-video-renderer.byline-separated > div#metadata > div#metadata-line > span"
channel_query = "ytd-channel-name#channel-name.long-byline,style-scope,ytd-video-renderer > div#container > div#text-container > yt-formatted-string.style-scope.ytd-channel-name > a"

In [10]:
# 데이터 추출 함수
def startSearch() :
    search_txt = input("search : ")
    encode_type = "utf-8"

    sq={"search_query":search_txt}
    search_encode = urlencode(sq)
    
    chrome = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    try:
        chrome.get("https://www.youtube.com/results?" + search_encode + "&sp=CAISAggF")
        i=0
        dataRuw=0
        isEnd=0
        time.sleep(1)

        # 초기 파일 생성
        now_date = datetime.datetime.now()
        cvsFileName = "youtube_search_" + search_txt + "_" + now_date.strftime("%Y-%m-%d") + ".csv"
        with open(cvsFileName, "w", newline="", encoding=encode_type) as f:
            colnames = ['number', 'title', 'like', 'time', "channel"]
            w = csv.writer(f)
            w.writerow(colnames)

        # 데이터 추출을 위한 루프
        while isEnd < sleepMaxCount:
            isEnd = 0
            items = chrome.find_elements_by_css_selector(item_query)
            item_len=items.__len__()

            # 최초 데이터 로드 될때까지 루프, 최대 sleepMaxCount 까지만 루프
            while item_len==0 and isEnd < sleepMaxCount:
                time.sleep(1)
                items = chrome.find_elements_by_css_selector(item_query)
                item_len = items.__len__()
                isEnd = isEnd + 1

            # 최대 sleepMaxCount 루프까지 진행 된게 아이라면 isEnd 초기화
            if isEnd < sleepMaxCount:
                isEnd = 0

            # 스크롤에 의한 데이터 로드 체크 및 추출 루프
            while item_len <= dataRuw and isEnd < sleepMaxCount:
                # 브라우저 스크롤 맨 아래로 이동
                chrome.execute_script("window.scrollTo(0, document.getElementById('content').scrollHeight);")
                time.sleep(1)
                items = chrome.find_elements_by_css_selector(item_query)
                item_len = items.__len__()
                with open(cvsFileName, "a", newline="", encoding=encode_type) as f:
                    w = csv.writer(f)
                    number = 0
                    for item in items[dataRuw:]:
                        number=number+1
                        title_txt = ""
                        like_txt = ""
                        time_txt = ""
                        channel_txt = ""
                        try:
                            title_txt = item.find_element_by_css_selector(title_query).text
                        except:
                            print("title_txt error")
                            title_txt = ""
                        
                        try:
                            like_txt = item.find_elements_by_css_selector(like_query)[0].text
                        except:
                            print("like_txt error")
                            like_txt = ""
                            
                        try:
                            time_txt = item.find_elements_by_css_selector(time_query)[1].text
                        except:
                            print("time_txt error")
                            time_txt = ""

                        try:
                            channel_txt = item.find_element_by_css_selector(channel_query).text
                        except:
                            print("channel_txt error")
                            channel_txt = ""

                        #if encode_type == "euc_kr":
                        #    title_txt = title_txt.encode('euc_kr','replace').decode("euc_kr")
                        data = [number, title_txt, like_txt, time_txt, channel_txt]
                        print(data)
                        w.writerow(data)
                isEnd = isEnd + 1
                time.sleep(1)

            dataRuw = item_len
            i = i+1
            print("스크롤 : ", i)
            print("데이터 건수 : ", dataRuw)
    except Exception as e:
        print(e)
    finally:
        if chrome:
            chrome.close()


In [11]:
startSearch()

search :  복권


[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\gbible\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


 
스크롤 :  1
데이터 건수 :  20
[1, '[동행복권] 로또 6/45 제961회 당첨결과!! 5등 1게임 & 수요일 자동 7게임중 6.5수 출!! ☜◈☞', '조회수 22회', '1일 전', '필우즈73']
[2, '새로나온 즉석복권 스피또 1000, 스피또 2000 모두 다 긁어보겠습니다.', '조회수 2천회', '1일 전', '일등돼지']
[3, '[실시간 온라인 제일복권 ] 도비쿠지가 오픈했습니다. 2021.05.03', '조회수 85회', '스트리밍 시간: 1일 전', '핑토너굴FingerToy-핑거토이']
[4, "[이슈톡] '복권 뭉치' 훔치려다 힘만 뺀 도둑 (2021.05.04/뉴스투데이/MBC)", '조회수 9.1천회', '1일 전', 'MBCNEWS']
[5, '21.05.04 Instant Lottery tickets 즉석복권 스피또1000 10장 복권당첨', '조회수 199회', '1일 전', '복권당첨']
[6, '오늘은 블박복권 11회차! 시작합니다! - 일산 히든토이', '조회수 113회', '스트리밍 시간: 1일 전', '어르티- 일산 히든토이']
[7, '962회 로또복권 1등당첨을 기원합니다. 우리 구독자님들 행운이 같이하길 빌어요', '조회수 7회', '1일 전', '덕유산백도사']
스크롤 :  2
데이터 건수 :  27
스크롤 :  3
데이터 건수 :  32
스크롤 :  4
데이터 건수 :  37
스크롤 :  5
데이터 건수 :  40
[1, '[로또복권 무료지급-1+1 무료행사] 매일 로또복권 1매+ 2천원 씩 무료로 받아 행운을 잡아보자', '조회수 29회', '2일 전', 'K-happy lounge']
time_txt error
[2, '21.05.01 추첨 Lotto lottery 로또 961회 10만원구입 복권당첨', '조회수 213회', '', '복권당첨']
[3, '태국 복권당첨됐어요! #เชียงใหม่ #คนเกาหลี #고민상담', '조회수 596회', '스트리밍 시간: 2일 전', '치앙마이서울

WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.93)
